In [1]:
import pandas as pd
from collections import deque
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint, ModelCheckpoint
import time
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
import keras
#import neptune
#from neptunecontrib.monitoring.keras import NeptuneMonitor
import os 
import warnings
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import shap

Using TensorFlow backend.


In [2]:
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [3]:

#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)


#neptune.init(api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMzk2YTM2NzktYzM2NC00MGE5LTk1NzAtMjlkZWE3ZjhhZDZjIn0=', project_qualified_name='magnusvlillehaug/tensorflow-keras-integration')
#neptune.create_experiment('tensorflow-keras-quickstart')


filnavn = 'hurtigsett'
SEQ_LEN = 30  # how long of a preceeding sequence to collect for RNN
FUTURE_PERIOD_PREDICT = 5  # how far into the future are we trying to predict?
EPOCHS = 1  # how many passes through our data
# how many batches? Try smaller batch if you're getting OOM (out of memory) errors.
BATCH_SIZE = 64


Model_result = []

for _ in range(1):
    #Pairs =["BTCUSDT", "BNBUSDT", "ETHUSDT","EOSUSDT"]
    Pairs = ["BTCUSDT","ETHUSDT","ETHBTC","DOTUSDT","LINKUSDT","EOSUSDT", "BNBUSDT"]
    for i in Pairs: 
        
        RATIO_TO_PREDICT = "BTCUSDT"
        NAME = f"{RATIO_TO_PREDICT}--{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

        def classify(current, future):
            if float(future) > float(current):
                return 1
            else:  # otherwise... it's a 0!
                return 0

        def preprocess_df(df):
            df = df.drop("future", 1)
            pd.set_option('use_inf_as_na', True)
            df.fillna(method='ffill', inplace=True)

            for col in df.columns:  # go through all of the columns
                if col != "target":  # normalize all ... except for the target itself!
                    df.fillna(method='ffill', inplace=True)
                    #df[col] = df[col].pct_change()  # pct change "normalizes" the different currencies (each crypto coin has vastly diff values, we're really more interested in the other coin's movements)
                    df.dropna(inplace=True)  # remove the nas created by pct_change
                    df[col] = preprocessing.scale(df[col].values)  # scale between 0 and 1.
           
            df.dropna(inplace=True)  # cleanup again... jic.


            df.dropna(inplace=True)  # cleanup again... jic.
            sequential_data = []  # this is a list that will CONTAIN the sequences
            # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in
            prev_days = deque(maxlen=SEQ_LEN)

            for i in df.values:  # iterate over the values
                prev_days.append([n for n in i[:-1]])  # store all but the target
                if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
                    # append those bad boys!
                    sequential_data.append([np.array(prev_days), i[-1]])

            random.shuffle(sequential_data)  # shuffle for good measure.

            buys = []  # list that will store our buy sequences and targets
            sells = []  # list that will store our sell sequences and targets

            for seq, target in sequential_data:  # iterate over the sequential data
                if target == 0:  # if it's a "not buy"
                    sells.append([seq, target])  # append to sells list
                elif target == 1:  # otherwise if the target is a 1...
                    buys.append([seq, target])  # it's a buy!

            random.shuffle(buys)  # shuffle the buys
            random.shuffle(sells)  # shuffle the sells!

            lower = min(len(buys), len(sells))  # what's the shorter length?

            # make sure both lists are only up to the shortest length.
            buys = buys[:lower]
            # make sure both lists are only up to the shortest length.
            sells = sells[:lower]

            sequential_data = buys+sells  # add them together
            # another shuffle, so the model doesn't get confused with all 1 class then the other.
            random.shuffle(sequential_data)
            x = []
            y = []


            for seq, target in sequential_data:  # going over our new sequential data
                x.append(seq)  # X is the sequences
                y.append(target)  # y is the targets/labels (buys vs sell/notbuy)
            return np.array(x), y  # return X and y...and make X a numpy array!

        dataset = f'{filnavn}.csv'

        #df = pd.read_csv(dataset, names=['btcusdt_timestamp', 'btcusdt_open', 'btcusdt_high', 'btcusdt_low', 'btcusdt_close', 'btcusdt_volume', 'btcusdt_EMA_5_1m', 'btcusdt_EMA_8_1m', 'btcusdt_EMA_12_1m', 'btcusdt_EMA_20_1m', 'btcusdt_EMA_50_1m', 'btcusdt_EMA_200_1m', 'btcusdt_up_band_1m', 'btcusdt_mid_band_1m', 'btcusdt_low_band_1m', 'btcusdt_MACD', 'btcusdt_MACD_signal', 'btcusdtMACD_histogram', 'btcusdt_RSI', 'ethusdt_open', 'ethusdt_high', 'ethusdt_low', 'ethusdt_close', 'ethusdt_volume', 'ethusdt_EMA_5_1m', 'ethusdt_EMA_8_1m', 'ethusdt_EMA_12_1m', 'ethusdt_EMA_20_1m', 'ethusdt_EMA_50_1m', 'ethusdt_EMA_200_1m', 'ethusdt_up_band_1m', 'ethusdt_mid_band_1m', 'ethusdt_low_band_1m', 'ethusdt_MACD', 'ethusdt_MACD_signal', 'ethusdtMACD_histogram', 'ethusdt_RSI', 'ethbtc_open', 'ethbtc_high', 'ethbtc_low', 'ethbtc_close', 'ethbtc_volume', 'ethbtc_EMA_5_1m', 'ethbtc_EMA_8_1m', 'ethbtc_EMA_12_1m', 'ethbtc_EMA_20_1m', 'ethbtc_EMA_50_1m', 'ethbtc_EMA_200_1m', 'ethbtc_up_band_1m', 'ethbtc_mid_band_1m', 'ethbtc_low_band_1m', 'ethbtc_MACD', 'ethbtc_MACD_signal', 'ethbtcMACD_histogram', 'ethbtc_RSI', 'uniusdt_open', 'uniusdt_high', 'uniusdt_low', 'uniusdt_close', 'uniusdt_volume', 'uniusdt_EMA_5_1m', 'uniusdt_EMA_8_1m', 'uniusdt_EMA_12_1m', 'uniusdt_EMA_20_1m', 'uniusdt_EMA_50_1m', 'uniusdt_EMA_200_1m', 'uniusdt_up_band_1m', 'uniusdt_mid_band_1m', 'uniusdt_low_band_1m', 'uniusdt_MACD', 'uniusdt_MACD_signal', 'uniusdtMACD_histogram', 'uniusdt_RSI', 'dotusdt_open', 'dotusdt_high', 'dotusdt_low', 'dotusdt_close', 'dotusdt_volume', 'dotusdt_EMA_5_1m', 'dotusdt_EMA_8_1m', 'dotusdt_EMA_12_1m', 'dotusdt_EMA_20_1m', 'dotusdt_EMA_50_1m', 'dotusdt_EMA_200_1m', 'dotusdt_up_band_1m', 'dotusdt_mid_band_1m', 'dotusdt_low_band_1m', 'dotusdt_MACD', 'dotusdt_MACD_signal', 'dotusdtMACD_histogram', 'dotusdt_RSI', 'linkusdt_open', 'linkusdt_high', 'linkusdt_low', 'linkusdt_close', 'linkusdt_volume', 'linkusdt_EMA_5_1m', 'linkusdt_EMA_8_1m', 'linkusdt_EMA_12_1m', 'linkusdt_EMA_20_1m', 'linkusdt_EMA_50_1m', 'linkusdt_EMA_200_1m', 'linkusdt_up_band_1m', 'linkusdt_mid_band_1m', 'linkusdt_low_band_1m', 'linkusdt_MACD', 'linkusdt_MACD_signal', 'linkusdtMACD_histogram', 'linkusdt_RSI', 'eosusdt_open', 'eosusdt_high', 'eosusdt_low', 'eosusdt_close', 'eosusdt_volume', 'eosusdt_EMA_5_1m', 'eosusdt_EMA_8_1m', 'eosusdt_EMA_12_1m', 'eosusdt_EMA_20_1m', 'eosusdt_EMA_50_1m', 'eosusdt_EMA_200_1m', 'eosusdt_up_band_1m', 'eosusdt_mid_band_1m', 'eosusdt_low_band_1m', 'eosusdt_MACD', 'eosusdt_MACD_signal', 'eosusdtMACD_histogram', 'eosusdt_RSI', 'bnbusdt_open', 'bnbusdt_high', 'bnbusdt_low', 'bnbusdt_close', 'bnbusdt_volume', 'bnbusdt_EMA_5_1m', 'bnbusdt_EMA_8_1m', 'bnbusdt_EMA_12_1m', 'bnbusdt_EMA_20_1m', 'bnbusdt_EMA_50_1m', 'bnbusdt_EMA_200_1m', 'bnbusdt_up_band_1m', 'bnbusdt_mid_band_1m', 'bnbusdt_low_band_1m', 'bnbusdt_MACD', 'bnbusdt_MACD_signal', 'bnbusdtMACD_histogram', 'bnbusdt_RSI'])
        main_df = pd.DataFrame()  # begin empty


         
        pd.set_option('use_inf_as_na', True)
        df = pd.read_csv(dataset)
        df.set_index("BNBUSDT_timestamp", inplace=True)
        #df = df[df[f"{pair}_close"].notna()]
        #df= df[[f"{pair}_close",f"{pair}_volume",f"{pair}_EMA_5_1m",f'{pair}_EMA_8_1m',f'{pair}_EMA_12_1m',f'{pair}_EMA_20_1m',f'{pair}_EMA_50_1m',f'{pair}_EMA_200_1m',f'{pair}_MACD',f'{pair}_MACD_signal',f'{pair}MACD_histogram',f'{pair}_RSI']]
        #df= df[[f"{pair}_close",f"{pair}_volume",f'{pair}_Signal_short',f'{pair}_Signal_medium',f'{pair}_Signal_long',f'{pair}_Signal_RSI_1m_bear',f'{pair}_Signal_RSI_1m_bull']]

        PAIR_TO_PREDICT = RATIO_TO_PREDICT
        df=df[[f'{PAIR_TO_PREDICT}_close',f'{PAIR_TO_PREDICT}_volume']]  

        #df= df[[f'{PAIR_TO_PREDICT}_MACD_sign_long',f'{PAIR_TO_PREDICT}_MACD_sign_short',f'{PAIR_TO_PREDICT}_Signal_RSI_1m_bull',f'{PAIR_TO_PREDICT}_Signal_ema_price_over_EMA50',f"{PAIR_TO_PREDICT}_close",f"{PAIR_TO_PREDICT}_volume",f'{i}_EMA_5_1m',f'{PAIR_TO_PREDICT}_EMA_8_1m',f'{PAIR_TO_PREDICT}_EMA_12_1m',f'{PAIR_TO_PREDICT}_EMA_20_1m',f'{PAIR_TO_PREDICT}_EMA_50_1m',f'{PAIR_TO_PREDICT}_EMA_200_1m',f'{PAIR_TO_PREDICT}_MACD',f'{PAIR_TO_PREDICT}_MACD_signal',f'{PAIR_TO_PREDICT}MACD_histogram',f'{PAIR_TO_PREDICT}_CCI_1m',f'{i}_MFI_1m',f'{PAIR_TO_PREDICT}_WILLR_1m']]
        
        df.fillna(method='ffill', inplace=True)
        
        if len(main_df) == 0:  # if the dataframe is empty
            main_df = df  # then it's just the current df
        else:  # otherwise, join this data to the main one
            main_df = main_df.join(df)

        main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
        main_df.dropna(inplace=True)
        main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)
        main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))
        main_df.to_csv('SHAP.csv')
        #print( main_df[[f"{RATIO_TO_PREDICT}_close_5m","future","target"]].head(10))

        main_df.dropna(inplace=True)

        # here, split away some slice of the future data from the main main_df.
        times = sorted(main_df.index.values)
     
        last_5pct = sorted(main_df.index.values)[-int(0.05*len(times))]
        
        validation_main_df = main_df[(main_df.index >= last_5pct)]

        main_df = main_df[(main_df.index < last_5pct)]

        train_x, train_y = preprocess_df(main_df)

        validation_x, validation_y = preprocess_df(validation_main_df)
        print(f"train data: {len(train_x)} validation: {len(validation_x)}")
        print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
        print(
            f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")


        train_x = np.asarray(train_x)
        train_y = np.asarray(train_y)
        validation_x = np.asarray(validation_x)
        validation_y = np.asarray(validation_y)

        model = Sequential()
        model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
        model.add(Dropout(0.5))
        model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

        model.add(LSTM(128, return_sequences=True))
        model.add(Dropout(0.5))
        model.add(BatchNormalization())

        model.add(LSTM(128))
        model.add(Dropout(0.5))
        model.add(BatchNormalization())

        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.2))

        model.add(Dense(2, activation='sigmoid'))

        opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

        #sparse_categorical_crossentropy
        # Compile model

        model.compile(
            loss='sparse_categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy']
        )
        #, km.binary_precision(), km.binary_recall()

        tensorboard = TensorBoard(log_dir="logs\\{}".format(NAME))

        # unique file name that will include the epoch and the validation acc for that epoch
        checkpoint_filepath = "models/RNN_Final-{epoch:02d}-.hd5"
        checkpoint = ModelCheckpoint(filepath=checkpoint_filepath , monitor='val_acc', verbose=1, save_best_only=True, mode='max')

train data: 8062 validation: 378
Dont buys: 4031, buys: 4031
VALIDATION Dont buys: 189, buys: 189
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
train data: 8062 validation: 378
Dont buys: 4031, buys: 4031
VALIDATION Dont buys: 189, buys: 189
train data: 8062 validation: 378
Dont buys: 4031, buys: 4031
VALIDATION Dont buys: 189, buys: 189
train data: 8062 validation: 378
Dont buys: 4031, buys: 4031
VALIDATION Dont buys: 189, buys: 189
train data: 8062 validation: 378
Dont buys: 4031, buys: 4031
VALIDATION Dont buys: 189, buys: 189
train data: 8062 validation: 378
Dont buys: 4031, buys: 4031
VALIDATION Dont buys: 189, buys: 189
train data: 8062 validation: 378
Dont buys: 4031, buys: 4031
VALIDATION Dont buys: 189, buys: 189


In [4]:
history = model.fit(
        train_x, train_y,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(validation_x, validation_y),
        callbacks=[tensorboard, checkpoint]
    )

score = model.evaluate(validation_x, validation_y, verbose=0)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 8062 samples, validate on 378 samples
8000/8062 [============================>.] - ETA: 0s - loss: 0.7184 - acc: 0.5109
Epoch 00001: val_acc improved from -inf to 0.50794, saving model to models/RNN_Final-01-.hd5
8062/8062 [==============================] - 119s 15ms/sample - loss: 0.7183 - acc: 0.5110 - val_loss: 0.6932 - val_acc: 0.5079


In [7]:
y_pr = model.predict(train_x)

In [8]:
print('Test utført på pair:', RATIO_TO_PREDICT)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test utført på pair: BTCUSDT
Test loss: 0.6931890797993493
Test accuracy: 0.50793654


In [9]:
variables_name = pd.read_csv('SHAP.csv')
features = variables_name.values[:,1]
features = list(variables_name.head(0))

In [10]:
Model_result = [{'Test loss' :score[0], 'Test accuracy:': score[1],'Trading par:':RATIO_TO_PREDICT  }]
#print("model outputs: {}".format(pred))
model.save("models/{}".format(NAME))

In [11]:
shap.initjs()

simple_model_only_first_output = tf.keras.Model(inputs=model.inputs,outputs=model.outputs[0],  # specifying a single output for shap usage
)

explainer = shap.DeepExplainer(simple_model_only_first_output, [train_x[:5], validation_x[:6]])




keras is no longer supported, please use tf.keras instead.


In [12]:
shap_values = explainer.shap_values(validation_x[:6])

In [14]:
shap.initjs()

shap.force_plot(explainer.expected_value[0], shap_values[0][0], validation_x[0])

